## TensorBoardサンプルコード

* tensorflow==1.13.1

on_epoch_endイベント時にTensorBoardにaccuracyを出力する。

### 簡易なモデルを記述（fashion-mnist）

In [2]:
import datetime
import numpy as np

import tensorflow as tf
from tensorflow import keras
from sklearn import model_selection
from tensorflow.keras import layers

# データの用意
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

# testデータの1/10をvalidationデータに使う
test_images, val_images, test_labels, val_labels = model_selection.train_test_split(
    test_images, test_labels, test_size=0.1, stratify=test_labels)

# 単純な推定器
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

## callbacksの記述

tf.keras.callbacks.TensorBoard を継承して on_epoch_end 時にsummaryに書き込む処理を実行する。

In [3]:
class MyCallback(tf.keras.callbacks.TensorBoard):
    
    def __init__(self, val_images, val_labels, test_images, test_labels, **kwargs):
        super().__init__(**kwargs)
        self.val_images = val_images
        self.val_labels = val_labels
        self.test_images = test_images
        self.test_labels = test_labels

    def on_epoch_end(self, epoch, logs={}):
        super().on_epoch_end(epoch, logs)
        
        # validationとtestのaccとlossを取得
        val_loss, val_acc = self.model.evaluate(self.val_images, self.val_labels, verbose=0)
        test_loss, test_acc = self.model.evaluate(self.test_images, self.test_labels, verbose=0)
        val_test_logs = {
            'epoch_end/val_loss': val_loss,
            'epoch_end/val_acc': val_acc,
            'epoch_end/test_loss': test_loss,
            'epoch_end/test_acc': test_acc
        }

        # summaryの設定
        values = [tf.Summary.Value(tag=name, simple_value=value)
            for name, value in val_test_logs.items()]
        summary = tf.Summary(value=values)
        self.writer.add_summary(summary, epoch)

## callbacksを指定してfit

自作のcallbackを指定してfitを実行。

In [4]:
my_callback = MyCallback(
    val_images, val_labels, test_images, test_labels,
    log_dir='logs')

model.fit(
    train_images,
    train_labels,
    epochs=50,
    callbacks=[my_callback])

Epoch 1/50
60000/60000 [==============================] - 5s 83us/sample - loss: 0.5026 - acc: 0.8236
Epoch 2/50
60000/60000 [==============================] - 5s 80us/sample - loss: 0.3777 - acc: 0.8635
Epoch 3/50
60000/60000 [==============================] - 5s 79us/sample - loss: 0.3391 - acc: 0.8767
Epoch 4/50
60000/60000 [==============================] - 5s 83us/sample - loss: 0.3154 - acc: 0.8846
Epoch 5/50
60000/60000 [==============================] - 5s 85us/sample - loss: 0.2986 - acc: 0.8899
Epoch 6/50
60000/60000 [==============================] - 5s 80us/sample - loss: 0.2805 - acc: 0.8958
Epoch 7/50
60000/60000 [==============================] - 5s 83us/sample - loss: 0.2685 - acc: 0.9003
Epoch 8/50
60000/60000 [==============================] - 5s 84us/sample - loss: 0.2587 - acc: 0.9034
Epoch 9/50
60000/60000 [==============================] - 5s 83us/sample - loss: 0.2477 - acc: 0.9088
Epoch 10/50
60000/60000 [==============================] - 5s 77us/sample - loss: 